In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
# from datasets import Dataset, DatasetDict
dataset_path = "text.csv"
df = pd.read_csv(dataset_path)
df = df.drop("Unnamed: 0", axis=1)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=10, shuffle=True)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# tokenizer.pad_token = tokenizer.eos_token
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# # train_tokenized_datasets = train_dataset.map(tokenize_function, batched = True)
# test_tokenized_datasets = test_dataset.map(tokenize_function, batched = True)

# # train_tokenized_datasets = train_tokenized_datasets.remove_columns(['text',  '__index_level_0__'])
# test_tokenized_datasets = test_tokenized_datasets.remove_columns(['text',  '__index_level_0__'])

# # train_tokenized_datasets.set_format(type='torch', columns = ['input_ids', 'attention_mask', 'label'])
# test_tokenized_datasets.set_format(type='torch', columns = ['input_ids', 'attention_mask', 'label'])
# test_tokenized_datasets

/home/zhang.yuchen/.conda/envs/env_cuda/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import GPT2Tokenizer
from transformers import GPT2ForSequenceClassification, GPT2Config
import torch

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [4]:
model = GPT2ForSequenceClassification.from_pretrained("model")
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)

In [5]:
from transformers import pipeline
pipe = pipeline("text-classification", model = model, tokenizer = tokenizer, device="cuda")
pipe

In [6]:
test_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 83362
})

In [7]:
test_strings = [d["text"] for d in test_dataset]
test_labels = [d["label"] for d in test_dataset]
test_strings

['i feel like im doomed until he returns',
 'i feel like until now you ve been having some pretty delicate personal problems',
 'i feel like a more valued employee',
 'i feel for you and it is divine and true specially meant for you',
 'i am pretty much one that tells you exactly how i am feeling even if the feelings are not so glamorous so here it goes',
 'i adore the homespun feel to leannes card with the loving hand stitching and the subtle distressing along with her old world quilt feel to the card the papers that she picked remind me so much of holly hobby d just gorgeous',
 'i still sort of feel like ive damaged my reputation after less than a week of being here',
 'i leave feeling distraught',
 'i start feel the pressure and having this weird feeling in my stomach gosh',
 'i feel honored to be able to give them this gift because it will bring them so much joy for years to come',
 'i feel so mad and humiliated myself whenever i tried to do it',
 'im way more than feeling like im 

In [9]:
test_pred = pipe(test_strings)
test_pred

[{'label': 'LABEL_0', 'score': 0.9999985694885254},
 {'label': 'LABEL_2', 'score': 0.9999927282333374},
 {'label': 'LABEL_1', 'score': 0.9999934434890747},
 {'label': 'LABEL_1', 'score': 0.9999972581863403},
 {'label': 'LABEL_1', 'score': 0.9999971389770508},
 {'label': 'LABEL_2', 'score': 0.9999892711639404},
 {'label': 'LABEL_0', 'score': 0.9999972581863403},
 {'label': 'LABEL_4', 'score': 0.9999985694885254},
 {'label': 'LABEL_5', 'score': 0.506069540977478},
 {'label': 'LABEL_1', 'score': 0.9999970197677612},
 {'label': 'LABEL_3', 'score': 0.999992847442627},
 {'label': 'LABEL_3', 'score': 0.6329602003097534},
 {'label': 'LABEL_1', 'score': 0.9999786615371704},
 {'label': 'LABEL_4', 'score': 0.9999978542327881},
 {'label': 'LABEL_1', 'score': 0.9999942779541016},
 {'label': 'LABEL_1', 'score': 0.9999951124191284},
 {'label': 'LABEL_1', 'score': 0.5176308751106262},
 {'label': 'LABEL_3', 'score': 0.9999960660934448},
 {'label': 'LABEL_0', 'score': 0.9999933242797852},
 {'label': 'LA

In [14]:
import json

with open('test_prediction.json', 'w') as fout:
    json.dump(pred, fout)

In [6]:
train_strings = [d["text"] for d in train_dataset]
train_labels = [d["label"] for d in train_dataset]
train_pred = pipe(train_strings)

In [8]:
import json

with open('train_prediction.json', 'w') as fout:
    json.dump(train_pred, fout)

In [2]:
import json
with open("test_prediction.json", "r") as read_file:
    test_pred = json.load(read_file)
with open("train_prediction.json", "r") as read_file:
    train_pred = json.load(read_file)

In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
# from datasets import Dataset, DatasetDict
dataset_path = "text.csv"
df = pd.read_csv(dataset_path)
df = df.drop("Unnamed: 0", axis=1)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=10, shuffle=True)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

test_labels = [d["label"] for d in test_dataset]

train_labels = [d["label"] for d in train_dataset]

In [11]:
from sklearn import metrics

train_pred_class = [int(p["label"][-1]) for p in train_pred]
test_pred_class = [int(p["label"][-1]) for p in test_pred]

train_acc = metrics.accuracy_score(train_labels, train_pred_class)
test_acc = metrics.accuracy_score(test_labels, test_pred_class)

print(f"Train accuracy: {train_acc}")
print(f"Test accuracy: {test_acc}")

Train accuracy: 0.9452716623631342
Test accuracy: 0.9372975696360452
